In [1]:


# año, torneo, nombre, edad,altura, ranking_ATP_inicial, minutos_jugados_acumulado_hist,
# pj_acum_historicos, pg_acum_historico, pp_acum_historico, titulos_ganados_acum, ultima_modificacion, posicion_final, score


# 2014,  roland_garros, rafa, 7, 1, 7, 0, sum(minutos), fijo, 6, clay, 1, 100, 2





# predecir 2015

# 1 -> 32/32=1
# 2 -> 31/32=0.99

In [2]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import pyprojroot
import sys

package_path = pyprojroot.here().__str__()
if package_path not in sys.path:
    sys.path.append(package_path)

In [3]:
clean_data = 'atp_matches_historic.csv'

In [4]:
from utils import LoadData
import pandas as pd

In [5]:
df = LoadData().from_csv(file_name=clean_data)
df.head(2)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2000-560,US Open,Hard,128.0,G,20000828.0,1.0,101736.0,1.0,NaN,...,3.0,3.0,8.0,90.0,52.0,30.0,13.0,12.0,5.0,12.0
1,2000-560,US Open,Hard,128.0,G,20000828.0,2.0,103096.0,NaN,NaN,...,3.0,5.0,4.0,84.0,53.0,34.0,12.0,13.0,7.0,12.0


In [6]:
df.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

In [7]:
def organize_data_type(data: pd.DataFrame, data_type: str):
    df = data.copy()

    cols = [
        "tourney_id",
        "tourney_name",
        "surface",
        "tourney_level",
        "tourney_date",
        f"{data_type}_id",
        f"{data_type}_name",
        f"{data_type}_ht",
        f"{data_type}_age",
        f"{data_type}_rank",
        "minutes",
        "round",
    ]

    rounds = df["round"].unique()
    change_round = {
        r: int(r.split("R")[-1])
        if r.startswith("R")
        else 4
        if r.startswith("Q")
        else 2
        if r.startswith("S")
        else 1
        if r.startswith("F")
        else 0
        for r in rounds
    }
    df["round"].replace(change_round, inplace=True)

    rename_cols = {col: col.split("_")[-1] for col in cols if col.startswith(data_type)}
    df["tourney_date"] = pd.to_datetime(df["tourney_date"])

    types = {
        "tourney_id": str,
        "tourney_name": str,
        "surface": str,
        "tourney_level": str,
        "id": str,
        "name": str,
        "ht": float,
        "age": float,
        "rank": float,
        "minutes": float,
    }

    return df.loc[:, cols].rename(columns=rename_cols).astype(types)


In [8]:
def organize_original_data(data):
    original_data = data.copy()

    original_data.tourney_date = pd.to_datetime(
        original_data.tourney_date.astype(int), format="%Y%m%d"
    )

    winner_df = organize_data_type(data=original_data, data_type="winner")
    winner_df["won"] = 1
    loser_df = organize_data_type(data=original_data, data_type="loser")
    loser_df["won"] = 0

    final_df = pd.concat([winner_df, loser_df]).rename(columns={"name": "player"})
    final_df.set_index("tourney_date", inplace=True)
    return final_df


final_df = organize_original_data(data=df)


In [9]:
def calc_historical_features(row, history):
    df = history.loc[
        (history["tourney_name"] == row["tourney_name"])
        & (history["player"] == row["player"]),
        ["tourney_name", "player", "won", "minutes"],
    ].copy()

    agg_dictionary = {"won": ["count", "sum"], "minutes": "sum"}

    df = df.groupby(by=["tourney_name", "player"]).agg(agg_dictionary).reset_index()
    df.columns = df.columns.map(lambda x: "".join(map(str, x)))
    df.rename(
        columns={"woncount": "pj", "wonsum": "pg", "minutessum": "minutes_played"},
        inplace=True,
    )
    df["pp"] = df["pj"] - df["pg"]

    return df


def make_features(data: pd.DataFrame, start_year: int = None, end_year: int = None):
    # Organize data
    organized_data = organize_original_data(data=data)

    if start_year is None:
        start_year = organized_data.index.year.min()
    if end_year is None:
        end_year = organized_data.index.year.max()

    final_data = pd.DataFrame()

    # Calculate features for each year
    for year in range(start_year, end_year + 1):
        print(year)

        # Current features
        current_tourneys = organized_data.sort_index().loc[str(year)]

        unique_cols = [
            "tourney_name",
            "tourney_level",
            "surface",
            "tourney_id",
            "player",
            "id",
        ]

        group = current_tourneys.groupby(by=unique_cols).agg(
            {"ht": "max", "age": "max", "rank": "max", "round": ["min"]}
        )

        current_features = group.reset_index()
        current_features.columns = current_features.columns.droplevel(1)

        # Historical features
        history = organized_data.sort_index().loc[: str(year - 1)]
        historical_features = pd.concat(
            current_features.apply(
                lambda row: calc_historical_features(row, history), axis=1
            ).to_list(),
            ignore_index=True,
        )

        full_features = current_features.merge(
            historical_features, how="left", on=["tourney_name", "player"]
        )
        full_features = full_features.fillna(
            value={"pj": 0, "pg": 0, "pp": 0, "minutes_played": 0}
        )
        full_features["year"] = year

        # Sort output data
        out_cols = [
            "year",
            "tourney_name",
            "tourney_level",
            "surface",
            "tourney_id",
            "player",
            "id",
            "ht",
            "age",
            "rank",
            "minutes_played",
            "pj",
            "pg",
            "pp",
            "round",
        ]

        final_data = pd.concat([final_data, full_features], ignore_index=True)

    return final_data.loc[:, out_cols]


data_features = make_features(data=df)


2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017


In [10]:
data_features

,year,tourney_name,tourney_level,surface,tourney_id,player,id,ht,age,rank,minutes_played,pj,pg,pp,round
0,2000,Australian Open,G,Hard,2000-580,Adrian Voinea,102437.0,185.0,25.448323,95.0,0.0,0.0,0.0,0.0,32
1,2000,Australian Open,G,Hard,2000-580,Albert Costa,102610.0,180.0,24.563997,20.0,0.0,0.0,0.0,0.0,128
2,2000,Australian Open,G,Hard,2000-580,Albert Portas,102287.0,188.0,26.171116,91.0,0.0,0.0,0.0,0.0,64
3,2000,Australian Open,G,Hard,2000-580,Alberto Berasategui,102238.0,173.0,26.554415,59.0,0.0,0.0,0.0,0.0,128
4,2000,Australian Open,G,Hard,2000-580,Alberto Martin,103252.0,175.0,21.409993,52.0,0.0,0.0,0.0,0.0,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,2017,Australian Open,G,Hard,2017-580,Tommy Haas,103163.0,188.0,38.789870,NaN,4593.0,32.0,21.0,11.0,128
8828,2017,Australian Open,G,Hard,2017-580,Victor Estrella,103607.0,173.0,36.457221,103.0,231.0,2.0,0.0,2.0,64
8829,2017,Australian Open,G,Hard,2017-580,Viktor Troicki,104678.0,193.0,30.932238,35.0,2740.0,17.0,9.0,8.0,32
8830,2017,Australian Open,G,Hard,2017-580,Yen Hsun Lu,104229.0,180.0,33.426420,60.0,2654.0,18.0,7.0,11.0,128


# Check integrality

In [11]:
year = 2017
player = 'Roger Federer'
tourney = 'Australian Open'

In [12]:
data_features[(data_features.player == player) & (data_features.tourney_name == tourney) & (data_features.year == year)]

,year,tourney_name,tourney_level,surface,tourney_id,player,id,ht,age,rank,minutes_played,pj,pg,pp,round
8812,2017,Australian Open,G,Hard,2017-580,Roger Federer,103819.0,185.0,35.441478,17.0,12211.0,94.0,81.0,13.0,1


In [13]:
# Current
final_df[(final_df.tourney_name ==tourney) & (final_df.index.year == year) & (final_df.player == player)]

,tourney_id,tourney_name,surface,tourney_level,id,player,ht,age,rank,minutes,round,won
tourney_date,,,,,,,,,,,,
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,125.0,128,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,123.0,64,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,90.0,32,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,203.0,16,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,92.0,4,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,184.0,2,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,217.0,1,1


In [14]:
# History
final_df[(final_df.tourney_name ==tourney) & (final_df.index.year < year) & (final_df.player == player)]

,tourney_id,tourney_name,surface,tourney_level,id,player,ht,age,rank,minutes,round,won
tourney_date,,,,,,,,,,,,
2000-01-17,2000-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,18.442163,62.0,125.0,128,1
2000-01-17,2000-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,18.442163,62.0,106.0,64,1
2001-01-15,2001-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,19.438741,29.0,132.0,128,1
2001-01-15,2001-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,19.438741,29.0,105.0,64,1
2002-01-14,2002-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,20.435318,12.0,126.0,128,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2012-01-16,2012-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,30.439425,3.0,222.0,2,0
2013-01-14,2013-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,31.436003,2.0,240.0,2,0
2014-01-13,2014-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,32.432580,6.0,144.0,2,0


In [15]:
final_df[(final_df['round'] == 1) & (final_df.won == 1)]

,tourney_id,tourney_name,surface,tourney_level,id,player,ht,age,rank,minutes,round,won
tourney_date,,,,,,,,,,,,
2000-08-28,2000-560,US Open,Hard,G,103498.0,Marat Safin,193.0,20.585900,7.0,98.0,1,1
2000-01-17,2000-580,Australian Open,Hard,G,101736.0,Andre Agassi,180.0,29.719370,1.0,139.0,1,1
2000-06-26,2000-540,Wimbledon,Grass,G,101948.0,Pete Sampras,185.0,28.873374,3.0,178.0,1,1
2000-05-29,2000-520,Roland Garros,Clay,G,102856.0,Gustavo Kuerten,190.0,23.715264,5.0,224.0,1,1
2001-01-15,2001-580,Australian Open,Hard,G,101736.0,Andre Agassi,180.0,30.715948,6.0,106.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-18,2016-580,Australian Open,Hard,G,104925.0,Novak Djokovic,188.0,28.659822,1.0,173.0,1,1
2016-05-23,2016-520,Roland Garros,Clay,G,104925.0,Novak Djokovic,188.0,29.004791,1.0,183.0,1,1
2016-06-27,2016-540,Wimbledon,Grass,G,104918.0,Andy Murray,190.0,29.119781,2.0,168.0,1,1
